In [1]:
import pypsa 
import atlite 
import vresutils.shapes as vshapes
from vresutils.graph import voronoi_partition_pts
import geopandas as gpd
import powerplantmatching as pm
import pandas as pd
from scipy.spatial import KDTree
import numpy as np
import geopandas as gpd

from pypsa.opt import Constraint as Con

import matplotlib.pyplot as plt
%matplotlib inline
import xarray as xr

In [2]:
!hostname

resi


In [4]:
n = pypsa.Network('./vietnam3_ninja_storage_PHS.nc')

INFO:pypsa.io:Imported network vietnam3_ninja_storage_PHS.nc has buses, carriers, generators, lines, loads, storage_units


In [5]:
n.storage_units

,bus,carrier,component,p_nom,max_hours,p_nom_extendable,p_nom_min,p_nom_max,marginal_cost,capital_cost,...,p_max_pu,p_set,q_set,sign,state_of_charge_initial,state_of_charge_set,cyclic_state_of_charge,standing_loss,inflow,p_nom_opt
name,,,,,,,,,,,,,,,,,,,,,
ChonThanh hydro,ChonThanh,hydro,Store,282.00,1.037106,True,282.00,423.000000,0.005,211.2,...,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
DaNang hydro,DaNang,hydro,Store,21.00,5.537143,True,21.00,31.500000,0.005,211.2,...,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
DakNong hydro,DakNong,hydro,Store,1781.50,0.332788,True,1781.50,2672.250000,0.005,211.2,...,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
DiLinh hydro,DiLinh,hydro,Store,1141.10,0.257549,True,1141.10,1711.650000,0.005,211.2,...,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
DocSoi hydro,DocSoi,hydro,Store,166.10,0.572173,True,166.10,249.150000,0.005,211.2,...,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
HaTinh hydro,HaTinh,hydro,Store,55.30,1.445859,True,55.30,82.950000,0.005,211.2,...,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
HiepHoa hydro,HiepHoa,hydro,Store,30.39,11.760563,True,30.39,45.585000,0.005,211.2,...,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
HoaBinh hydro,HoaBinh,hydro,Store,615.10,1.462422,True,615.10,922.650000,0.005,211.2,...,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0
LaiChau hydro,LaiChau,hydro,Store,464.10,0.124415,True,464.10,696.150000,0.005,211.2,...,1.0,0.0,0.0,1.0,0.0,NaN,False,0.0,0.0,0.0


In [6]:
n.storage_units.loc[n.buses.index+' hydro'].efficiency_dispatch

HiepHoa hydro       0.9
LaiChauH hydro      0.9
DongAnh hydro       0.9
SonLa hydro         0.9
SonLaH hydro        0.9
MongDuong1 hydro    0.9
MongDuong2 hydro    0.9
QuangNinh hydro     0.9
NghiSon hydro       0.9
HoaBinh hydro       0.9
ThangLongT hydro    0.9
ThuongTin hydro     0.9
PhoNoi hydro        0.9
NhoQuan hydro       0.9
HaTinh hydro        0.9
VungAng hydro       0.9
DaNang hydro        0.9
DocSoi hydro        0.9
ThanhMy hydro       0.9
Pleiku hydro        0.9
Pleiku2 hydro       0.9
Yali hydro          0.9
DakNong hydro       0.9
DiLinh hydro        0.9
CauBong hydro       0.9
TanDinh hydro       0.9
SongMay hydro       0.9
VinhTan hydro       0.9
PhuLam hydro        0.9
MyTho hydro         0.9
DuyenHai hydro      0.9
ChonThanh hydro     0.9
OMon hydro          0.9
CuChi hydro         0.9
NhaBe hydro         0.9
PhuMy hydro         0.9
LaiChau hydro       0.9
TayHaNoi hydro      0.9
VietTri hydro       0.9
VanPhong hydro      0.9
CongThanh hydro     0.9
NamDinh1 hydro  

In [7]:
n.storage_units.loc[n.buses.index+' hydro'].efficiency_store

HiepHoa hydro       0.9
LaiChauH hydro      0.9
DongAnh hydro       0.9
SonLa hydro         0.9
SonLaH hydro        0.9
MongDuong1 hydro    0.9
MongDuong2 hydro    0.9
QuangNinh hydro     0.9
NghiSon hydro       0.9
HoaBinh hydro       0.9
ThangLongT hydro    0.9
ThuongTin hydro     0.9
PhoNoi hydro        0.9
NhoQuan hydro       0.9
HaTinh hydro        0.9
VungAng hydro       0.9
DaNang hydro        0.9
DocSoi hydro        0.9
ThanhMy hydro       0.9
Pleiku hydro        0.9
Pleiku2 hydro       0.9
Yali hydro          0.9
DakNong hydro       0.9
DiLinh hydro        0.9
CauBong hydro       0.9
TanDinh hydro       0.9
SongMay hydro       0.9
VinhTan hydro       0.9
PhuLam hydro        0.9
MyTho hydro         0.9
DuyenHai hydro      0.9
ChonThanh hydro     0.9
OMon hydro          0.9
CuChi hydro         0.9
NhaBe hydro         0.9
PhuMy hydro         0.9
LaiChau hydro       0.9
TayHaNoi hydro      0.9
VietTri hydro       0.9
VanPhong hydro      0.9
CongThanh hydro     0.9
NamDinh1 hydro  

In [8]:
n.storage_units.loc[n.buses.index+' hydro', 'efficiency_store'] = 0.0

In [9]:
n.storage_units.loc[n.buses.index+' hydro'].efficiency_store

HiepHoa hydro       0.0
LaiChauH hydro      0.0
DongAnh hydro       0.0
SonLa hydro         0.0
SonLaH hydro        0.0
MongDuong1 hydro    0.0
MongDuong2 hydro    0.0
QuangNinh hydro     0.0
NghiSon hydro       0.0
HoaBinh hydro       0.0
ThangLongT hydro    0.0
ThuongTin hydro     0.0
PhoNoi hydro        0.0
NhoQuan hydro       0.0
HaTinh hydro        0.0
VungAng hydro       0.0
DaNang hydro        0.0
DocSoi hydro        0.0
ThanhMy hydro       0.0
Pleiku hydro        0.0
Pleiku2 hydro       0.0
Yali hydro          0.0
DakNong hydro       0.0
DiLinh hydro        0.0
CauBong hydro       0.0
TanDinh hydro       0.0
SongMay hydro       0.0
VinhTan hydro       0.0
PhuLam hydro        0.0
MyTho hydro         0.0
DuyenHai hydro      0.0
ChonThanh hydro     0.0
OMon hydro          0.0
CuChi hydro         0.0
NhaBe hydro         0.0
PhuMy hydro         0.0
LaiChau hydro       0.0
TayHaNoi hydro      0.0
VietTri hydro       0.0
VanPhong hydro      0.0
CongThanh hydro     0.0
NamDinh1 hydro  

In [10]:
n.storage_units.loc[n.buses.index+' hydro'].efficiency_dispatch

HiepHoa hydro       0.9
LaiChauH hydro      0.9
DongAnh hydro       0.9
SonLa hydro         0.9
SonLaH hydro        0.9
MongDuong1 hydro    0.9
MongDuong2 hydro    0.9
QuangNinh hydro     0.9
NghiSon hydro       0.9
HoaBinh hydro       0.9
ThangLongT hydro    0.9
ThuongTin hydro     0.9
PhoNoi hydro        0.9
NhoQuan hydro       0.9
HaTinh hydro        0.9
VungAng hydro       0.9
DaNang hydro        0.9
DocSoi hydro        0.9
ThanhMy hydro       0.9
Pleiku hydro        0.9
Pleiku2 hydro       0.9
Yali hydro          0.9
DakNong hydro       0.9
DiLinh hydro        0.9
CauBong hydro       0.9
TanDinh hydro       0.9
SongMay hydro       0.9
VinhTan hydro       0.9
PhuLam hydro        0.9
MyTho hydro         0.9
DuyenHai hydro      0.9
ChonThanh hydro     0.9
OMon hydro          0.9
CuChi hydro         0.9
NhaBe hydro         0.9
PhuMy hydro         0.9
LaiChau hydro       0.9
TayHaNoi hydro      0.9
VietTri hydro       0.9
VanPhong hydro      0.9
CongThanh hydro     0.9
NamDinh1 hydro  

In [11]:
n.storage_units.efficiency_store

name
ChonThanh hydro       0.00
DaNang hydro          0.00
DakNong hydro         0.00
DiLinh hydro          0.00
DocSoi hydro          0.00
HaTinh hydro          0.00
HiepHoa hydro         0.00
HoaBinh hydro         0.00
LaiChau hydro         0.00
LaiChauH hydro        0.00
NghiSon hydro         0.00
NhoQuan hydro         0.00
Pleiku hydro          0.00
Pleiku2 hydro         0.00
SonLa hydro           0.00
SonLaH hydro          0.00
SongMay hydro         0.00
TayHaNoi hydro        0.00
ThanhMy hydro         0.00
VanPhong hydro        0.00
VinhTan hydro         0.00
Yali hydro            0.00
CauBong battery       0.90
ChonThanh battery     0.90
CongThanh battery     0.90
CuChi battery         0.90
DaNang battery        0.90
DakNong battery       0.90
DiLinh battery        0.90
DocSoi battery        0.90
                      ... 
ThuongTin hydrogen    0.75
VanPhong hydrogen     0.75
VietTri hydrogen      0.75
VinhTan hydrogen      0.75
VungAng hydrogen      0.75
Yali hydrogen         0

In [12]:
n.storage_units.efficiency_dispatch

name
ChonThanh hydro       0.90
DaNang hydro          0.90
DakNong hydro         0.90
DiLinh hydro          0.90
DocSoi hydro          0.90
HaTinh hydro          0.90
HiepHoa hydro         0.90
HoaBinh hydro         0.90
LaiChau hydro         0.90
LaiChauH hydro        0.90
NghiSon hydro         0.90
NhoQuan hydro         0.90
Pleiku hydro          0.90
Pleiku2 hydro         0.90
SonLa hydro           0.90
SonLaH hydro          0.90
SongMay hydro         0.90
TayHaNoi hydro        0.90
ThanhMy hydro         0.90
VanPhong hydro        0.90
VinhTan hydro         0.90
Yali hydro            0.90
CauBong battery       0.90
ChonThanh battery     0.90
CongThanh battery     0.90
CuChi battery         0.90
DaNang battery        0.90
DakNong battery       0.90
DiLinh battery        0.90
DocSoi battery        0.90
                      ... 
ThuongTin hydrogen    0.58
VanPhong hydrogen     0.58
VietTri hydrogen      0.58
VinhTan hydrogen      0.58
VungAng hydrogen      0.58
Yali hydrogen         0

In [13]:
n.export_to_netcdf('./temp/vietnam3_ninja_storage.nc')

INFO:pypsa.io:Exported network vietnam3_ninja_storage.nc has buses, carriers, lines, loads, generators, storage_units


<xarray.Dataset>
Dimensions:                            (buses_i: 46, carriers_i: 12, generators_i: 414, generators_t_p_max_pu_i: 138, lines_i: 65, loads_i: 46, loads_t_p_set_i: 46, snapshots: 17544, storage_units_i: 138, storage_units_t_inflow_i: 46)
Coordinates:
  * snapshots                          (snapshots) datetime64[ns] 2015-01-01 ...
  * buses_i                            (buses_i) object 'HiepHoa' 'LaiChauH' ...
  * carriers_i                         (carriers_i) object 'bioenergy' ...
  * lines_i                            (lines_i) object '1' '2' '3' '4' '5' ...
  * loads_i                            (loads_i) object 'HiepHoa' 'LaiChauH' ...
  * loads_t_p_set_i                    (loads_t_p_set_i) object 'HiepHoa' ...
  * generators_i                       (generators_i) object 'CongThanh hard coal' ...
  * generators_t_p_max_pu_i            (generators_t_p_max_pu_i) object 'CauBong hydro ror' ...
  * storage_units_i                    (storage_units_i) object 'ChonThanh h